# 🎓 LangSmith Prompt Pipeline 课程演示

> 简洁版演示 Notebook - 适合快速上手和课程演示

本教程将演示：
- ✅ 测试 LLM 连接和监控
- ✅ 提示词开发、评估与迭代完整工作流
- ✅ 复杂 Graph 场景中的数据跟踪挑战
- ✅ Dataset 捕获：解决复杂场景数据管理痛点


## 📦 1. 环境准备

### ⚠️ 重要提示

**使用 LangSmith 追踪功能，必须按顺序执行以下步骤：**

1. **运行 Cell 2** - 导入所有模块
2. **运行 Cell 3** - 启用 LangSmith 追踪（每次启动 notebook 都要运行）
3. 然后才能运行其他带追踪的 cell

**如果看不到追踪记录：**
- 检查 Cell 3 是否运行成功
- 检查输出中 `LANGCHAIN_TRACING_V2` 是否为 `true`
- Kernel 重启后需要重新运行 Cell 2 和 Cell 3

---

### 环境变量配置

确保 `.env` 文件配置正确：

```bash
# LangSmith（可选）
LANGSMITH_API_KEY="lsv2_pt_xxxx"
LANGSMITH_PROJECT="economic_report"

# Azure OpenAI
LLM_PROVIDER="azure"
AZURE_ENDPOINT="https://xxx.openai.azure.com/"
AZURE_DEPLOYMENT="gpt-4"
AZURE_API_KEY="your-key"
```


In [9]:
# 导入必要模块
import sys
from config.langsmith_config import LangSmithConfig
from config.azure_config import AzureConfig
from graph.graph import ReportGraphBuilder
from prompts.prompt_manager import PromptManager
from evaluation.datasets import DatasetManager
from tools.capture import capture_inputs, list_all_captures, sync_all_captures

print("✓ 模块导入成功")


✓ 模块导入成功


---

## 🔧 2. LLM 测试与监控

### 2.1 基础 LLM 测试


In [10]:
# 导入必要模块（如果单独运行此 cell）
from config.azure_config import AzureConfig

# 创建 LLM 实例
llm = AzureConfig.get_llm()

# 简单测试
response = llm.invoke("你好，请简单介绍一下什么是 LangSmith？")
print(f"✓ LLM 响应:\n{response.content}")


✓ LLM 响应:
LangSmith 是由 LangChain 团队推出的一个平台，旨在帮助开发者更高效地构建、评估和部署由语言模型驱动的应用程序。它提供了一套工具，可以让开发者在实际环境中进行模型的调试和优化，同时简化了与不同模型和数据源的集成过程。通过 LangSmith，开发者可以更轻松地管理和监控其应用程序的性能，确保模型输出的质量和可靠性。这使得构建复杂的自然语言处理应用变得更加容易和高效。


### 2.2 带监控的 LLM 调用

使用 `@traceable` 装饰器自动追踪 LLM 调用：


In [11]:
from langsmith import traceable
from config.azure_config import AzureConfig

@traceable(name="economic_report")
def test_llm_call(query: str):
    """带监控的 LLM 调用示例"""
    llm = AzureConfig.get_llm()
    response = llm.invoke(query)
    return response.content

# 测试
result = test_llm_call("请用一句话解释什么是 LangGraph？")
print(f"\n响应: {result}")
print("\n💡 访问 LangSmith 控制台查看完整追踪链路")



响应: LangGraph 是一个用于构建和操作自然语言处理任务的工具或框架，通常用于实现复杂的语言模型和对话系统。

💡 访问 LangSmith 控制台查看完整追踪链路


---

## 📝 3. 提示词开发与迭代

本章将演示提示词从测试、评估到迭代优化的完整工作流：

1. **测试单个提示词** - 查看和测试可用提示词
2. **使用评估器评估质量** - 量化提示词质量
3. **迭代优化提示词** - 修改、评估、推送完整流程

---

### 3.1 测试单个提示词

#### 查看可用的提示词


In [6]:
# 导入必要模块（如果单独运行此 cell）
from prompts.prompt_manager import PromptManager

# 创建 Prompt 管理器
prompt_manager = PromptManager()

# 列出所有提示词
print("可用的提示词:\n")
for name, config in prompt_manager.config.get('prompts', {}).items():
    print(f"  📝 {name}")
    print(f"     文件: {config.get('file')}")
    if config.get('description'):
        print(f"     描述: {config.get('description')}")
    print()


可用的提示词:

  📝 parameter_parser
     文件: parameter_parser.yaml
     描述: 从用户输入中提取结构化参数

  📝 report_generator
     文件: report_generator.yaml
     描述: 生成结构化的分析报告



#### 测试参数解析提示词


In [12]:
# 导入必要模块（如果单独运行此 cell）
from prompts.prompt_manager import PromptManager
from config.azure_config import AzureConfig

# 获取参数解析提示词
prompt_manager = PromptManager()
config = prompt_manager.get('parameter_parser')  # 获取配置字典
parameter_prompt = prompt_manager.create_prompt(config)  # 创建 Prompt 对象
print(parameter_prompt)
# 测试输入
test_query = "写一份关于人工智能行业2024年的正式详细分析报告，重点关注技术创新和市场趋势"

print(f"测试查询: {test_query}\n")

# 创建 Chain
llm = AzureConfig.get_llm()
chain = parameter_prompt | llm

# 调用
result = chain.invoke({"user_query": test_query})

print("解析结果:")
print(result.content)

[*] 检查远程更新: parameter_parser...
[OK] 已从 Hub 同步最新版本
input_variables=['user_query'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='你是一个专业的参数提取助手，擅长从自然语言中识别和提取结构化信息。\n\n## 你的能力：\n- 准确理解用户的意图和需求\n- 从模糊的描述中提取明确的参数\n- 处理缺失参数时使用合理的默认值\n- 输出标准格式的 JSON 数据\n\n## 行为准则：\n- 只返回纯 JSON，不要任何额外的解释或格式标记\n- 如果参数不明确，使用预定义的默认值\n- 确保输出的 JSON 格式正确，可以被直接解析\n- 不要臆测用户没有提到的信息\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_query'], input_types={}, partial_variables={}, template='请从以下用户输入中提取报告生成所需的参数：\n\n**用户输入**: {user_query}\n\n## 需要提取的参数：\n\n1. **topic** (必填): 报告主题\n2. **year_range** (可选): 数据年份范围\n   - 格式: "2023-2024" 或 "2023"\n   - 默认值: "2023-2024"\n\n3. **style** (可选): 编写风格\n   - 可选值: formal(正式), casual(轻松), detailed(详细), concise(简洁)\n   - 默认值: "formal"\n\n4. **depth** (可选): 分析深度\n   - 可选值: shallow(浅层), medium(中等), deep(深度)\n   - 默认值: "medium

#### 测试报告生成提示词


---

### 3.2 使用评估器评估提示词质量

测试提示词后，我们需要**量化评估**提示词的质量。评估器（Evaluator）可以自动评估提示词生成内容的质量。

#### 什么是评估器？

评估器用于**自动评估**提示词生成内容的质量，包括：
- 结构完整性
- 内容相关性
- 参数使用情况
- 整体质量分数

#### 查看可用的评估器


In [3]:
from evaluation.evaluators.report import ReportEvaluators

# 查看所有可用的评估器
print("可用的报告评估器:\n")
print("1. structure_evaluator - 检查报告结构")
print("2. content_completeness_evaluator - 检查内容完整性")
print("3. relevance_evaluator - 检查相关性")
print("4. parameter_usage_evaluator - 检查参数使用")


可用的报告评估器:

1. structure_evaluator - 检查报告结构
2. content_completeness_evaluator - 检查内容完整性
3. relevance_evaluator - 检查相关性
4. parameter_usage_evaluator - 检查参数使用


#### 运行评估


In [7]:
# 使用 PromptManager 的 evaluate_prompt 方法
prompt_manager = PromptManager()

# 评估 parameter_parser 提示词
print("🔍 开始评估 parameter_parser 提示词...\n")

result = prompt_manager.evaluate_prompt(
    prompt_name="parameter_parser",
    evaluators=[
        ReportEvaluators.structure_evaluator,
        ReportEvaluators.content_completeness_evaluator
    ]
)

print(f"\n📊 评估结果:")
print(f"  测试用例数: {result['total']}")
print(f"  质量分数: {result['quality_score']:.2%}")
print(f"\n详细分数:")
for eval_name, score in result.get('scores', {}).items():
    print(f"  - {eval_name}: {score:.2%}")


🔍 开始评估 parameter_parser 提示词...

[EVAL] 评估提示词: parameter_parser
  使用自定义评估器: 2 个

LangSmith Evaluator - 提示词质量评估

✓ 数据集 'parameter_parser' 已存在
提示词: parameter_parser
数据集: parameter_parser
实验名称: parameter_parser_evaluation
评估器数量: 2

开始评估...

[*] 检查远程更新: parameter_parser...
[OK] 已从 Hub 同步最新版本
View the evaluation results for experiment: 'parameter_parser_evaluation-9b581755' at:
https://smith.langchain.com/o/0e9cda67-5514-5e66-826a-519be74d85b9/datasets/f41ee129-87a6-4c44-90c8-f4ed1f7fee5d/compare?selectedSessions=9d6918ae-6c9c-49f5-a470-4db3a1e8f62d




2it [00:02,  1.19s/it]


评估完成！

评估结果汇总:
------------------------------------------------------------
  structure_valid: 85.00%
  content_complete: 90.00%
  relevance: 88.00%
  parameter_usage: 92.00%
------------------------------------------------------------
  总分: 88.75%
  测试数: 5

查看详细结果: https://smith.langchain.com/
  [OK] 评估完成
     - 测试用例: 5
     - 质量分数: 88.75%

📊 评估结果:
  测试用例数: 5
  质量分数: 88.75%

详细分数:
  - structure_valid: 85.00%
  - content_complete: 90.00%
  - relevance: 88.00%
  - parameter_usage: 92.00%


---

### 3.3 迭代优化提示词

评估完成后，如果质量不达标，我们需要修改提示词并重新评估，直到质量达标后再推送到 LangSmith Hub。

#### 完整迭代流程

```
1. 修改本地提示词 YAML 文件
2. 使用 PromptManager 评估新提示词
3. 如果质量达标，推送到 LangSmith Hub
4. 查看 LangSmith 上的评估结果
```

#### 步骤 1: 修改提示词（示例）

假设我们要优化 `parameter_parser` 提示词，在 `prompts/parameter_parser.yaml` 中修改后...

> 💡 **提示**：实际修改提示词时，请直接编辑 YAML 文件，这里只是演示流程。

#### 步骤 2: 评估修改后的提示词


In [ ]:
# 重新加载并评估（假设你已经修改了 YAML 文件）
prompt_manager = PromptManager(auto_pull=False)  # 不自动拉取，使用本地版本

# 评估本地修改的提示词
print("🔍 评估本地修改的提示词...\n")

eval_result = prompt_manager.evaluate_prompt("parameter_parser")

print(f"\n📊 评估结果:")
print(f"  质量分数: {eval_result['quality_score']:.2%}")

# 判断是否达标
MIN_SCORE = 0.80
if eval_result['quality_score'] >= MIN_SCORE:
    print(f"  ✅ 质量达标（≥ {MIN_SCORE:.0%}），可以推送")
else:
    print(f"  ❌ 质量不达标（< {MIN_SCORE:.0%}），需要继续优化")


#### 步骤 3: 推送到 LangSmith Hub（如果质量达标）


In [ ]:
# 推送到 Hub（包含自动评估和版本备份）
# 注意：只有在质量达标时才执行

if eval_result['quality_score'] >= MIN_SCORE:
    print("🚀 开始推送到 LangSmith Hub...\n")
    
    success = prompt_manager.push(
        prompt_name="parameter_parser",
        with_test=True,        # 推送前再次测试
        create_backup=True     # 创建版本备份
    )
    
    if success:
        print("\n✓ 推送成功！")
        print("📊 访问 LangSmith Hub 查看:")
        print("   https://smith.langchain.com/hub")
else:
    print("⚠️ 质量未达标，跳过推送")


In [ ]:
# 导入必要模块（如果单独运行此 cell）
from prompts.prompt_manager import PromptManager
from config.azure_config import AzureConfig

# 获取报告生成提示词
prompt_manager = PromptManager()
config = prompt_manager.get('report_generator')  # 获取配置字典
report_prompt = prompt_manager.create_prompt(config)  # 创建 Prompt 对象
print(str(report_prompt))
# 模拟输入参数
inputs = {
    "topic": "人工智能行业发展分析",
    "year_range": "2024",
    "style": "formal",
    "depth": "medium",
    "focus_areas": "技术创新,市场趋势",
    "search_results": """相关资料摘要：
    - 2024年人工智能市场规模预计达到5000亿美元
    - 生成式AI技术快速发展，ChatGPT用户突破2亿
    - 企业AI应用率从30%提升至60%
    """
}

# 创建 Chain 并调用
llm = AzureConfig.get_llm()
chain = report_prompt | llm
result = chain.invoke(inputs)

print("\n" + "="*60)
print("生成的报告:")
print("="*60)
print(result.content)


---

## 🔄 4. 运行完整 Graph 流程

### 4.1 快速生成报告


In [13]:
# 导入必要模块（如果单独运行此 cell）
from graph.graph import ReportGraphBuilder

# 创建 Graph 构建器
builder = ReportGraphBuilder()

# 运行
user_query = "写一份关于区块链技术2024年应用趋势的分析报告"

result = builder.run(user_query=user_query)

# 查看结果
print("\n" + "="*60)
print("生成的报告:")
print("="*60)
print(result.get('report', '')[:500] + "...\n")

# 查看元数据
print("元数据:")
for key, value in result.get('metadata', {}).items():
    print(f"  {key}: {value}")


[OK] Perplexity SDK 初始化成功
✓ LangGraph 工作流构建完成
  节点: parse_parameters -> web_search -> generate_report -> quality_check

开始运行报告生成流程
用户查询: 写一份关于区块链技术2024年应用趋势的分析报告

[节点] 解析参数...
[*] 检查远程更新: parameter_parser...
[OK] 已从 Hub 同步最新版本
  使用 Prompt: parameter_parser hub-20251101-152830
[Capture] 已保存: parameter_parser → run_20251101_152832_039018.json
[Sync] 已同步到 LangSmith: parameter_parser

[节点] 执行 Perplexity 搜索...
  [SEARCH] 使用 Perplexity Search API: 区块链技术2024年应用趋势 2023-2024 市场趋势,技术发展,竞争格局 分析报告 最新数据
[OK] Perplexity 搜索完成: 获取 3 条结果
[*] 检查远程更新: report_generator...
[OK] 已从 Hub 同步最新版本
[Capture] 已保存: report_generator → run_20251101_152858_447522.json
[Sync] 已同步到 LangSmith: report_generator

[节点] 质量检查...
✓ 质量检查通过

流程执行完成
✓ 报告生成成功

元数据:
  report_length: 2322
  prompt_version: hub-20251101-152844
  quality_checks: {'length': True, 'has_title': True, 'has_content': True}
  quality_passed: True

生成的报告:
## 一、引言（报告概述）

### 1.1 编制背景

党的二十大报告明确提出要加快发展数字经济，促进数字经济与实体经济深度融合。区块链技术作为数字经济的重要基础设施，正逐步成为推动产业变革和提升经济效益的

### 4.2 观察 LangSmith 追踪链路

运行上面的 Graph 后，访问 **LangSmith 控制台**，你会看到：

- **复杂的追踪树**：多个节点的嵌套调用
- **多个提示词调用**：`parameter_parser` 和 `report_generator` 都被调用
- **参数传递链**：从 `user_query` → 解析参数 → 生成报告的完整流程
- **每个节点的输入输出**：大量需要分析的数据

---

### 4.3 复杂场景的数据挑战

在刚才的 Graph 运行中，你看到了什么？

**数据追踪的复杂性：**

1. **多节点调用** - 一个 Graph 包含 4 个节点，每个都可能调用 LLM
2. **参数转换** - 参数在不同节点间转换（`user_query` → `topic`, `style` 等）
3. **分散的追踪数据** - LangSmith 追踪显示所有调用，但数据分散在不同节点
4. **真实场景数据** - 这些是真实的使用场景，包含实际的用户查询和参数

**痛点问题：**

❓ **如何系统性地收集这些真实使用场景的数据？**

- 手动复制粘贴？太麻烦且容易遗漏
- 从 LangSmith 导出？格式复杂且难以批量处理
- 缺乏结构化存储？无法用于后续评估和优化

💡 **这就是我们需要 Dataset 捕获功能的原因！**

---

### 4.4 带参数的生成（可选）

你也可以直接传入参数来运行 Graph：


In [ ]:
# 导入必要模块（如果单独运行此 cell）
from graph.graph import ReportGraphBuilder

# 创建构建器（如果之前没有创建）
if 'builder' not in locals():
    builder = ReportGraphBuilder()

# 带详细参数运行
result = builder.run(
    user_query="金融科技行业分析",
    year_range="2024",
    style="formal",
    depth="deep",
    focus_areas="技术创新,监管政策"
)

print("提取的参数:")
print(f"  主题: {result.get('topic')}")
print(f"  年份: {result.get('year_range')}")
print(f"  风格: {result.get('style')}")
print(f"  深度: {result.get('depth')}")
print(f"  关注领域: {result.get('focus_areas')}")

print("\n报告预览:")
print(result.get('report', '')[:300] + "...")
# 导入必要模块（如果单独运行此 cell）
from graph.graph import ReportGraphBuilder

# 创建构建器（如果之前没有创建）
if 'builder' not in locals():
    builder = ReportGraphBuilder()

# 异步模式可以看到每个节点的执行过程
result = await builder.arun(
    user_query="人工智能医疗应用2024年发展趋势"
)

print("\n最终报告:")
print(result.get('report', '')[:400] + "...")


---

## 💾 5. Dataset 捕获：解决复杂场景数据管理痛点

在 Graph 复杂场景中，我们看到了数据追踪的挑战。现在让我们学习如何使用 **Dataset 捕获功能**来系统性地收集和管理这些真实使用场景的数据。

### 5.1 为什么需要 Dataset 捕获？

在复杂 Graph 场景中，我们面临的数据管理问题：

1. **数据分散** - 多个节点、多个提示词调用，数据分散在不同位置
2. **真实场景难以收集** - 真实用户查询和参数难以手动收集
3. **评估需要数据** - 要评估提示词质量，需要真实的测试数据集
4. **持续优化** - 每次优化提示词都需要新的测试数据

**Dataset 捕获功能**可以：
- ✅ 自动捕获 Graph 节点中的 LLM 调用参数
- ✅ 记录真实使用场景
- ✅ 构建结构化的测试数据集
- ✅ 批量同步到 LangSmith Dataset，用于评估

---

### 5.2 查看已捕获的数据

在 Graph 运行时，我们已经看到了一些捕获信息：
```
[Capture] 已保存: parameter_parser → run_20251101_152832_039018.json
[Sync] 已同步到 LangSmith: parameter_parser
```

让我们查看当前所有已捕获的数据：


In [ ]:
# 导入必要模块（如果单独运行此 cell）
from tools.capture import list_all_captures

# 列出所有捕获的测试数据（显示详细信息）
captures = list_all_captures(verbose=True)

print(f"\n💡 当前共有 {len(captures)} 条捕获数据")


---

### 5.3 使用 @capture_dataset 装饰器

`@capture_dataset` 装饰器可以在运行 Graph 节点时**自动捕获** LLM 的输入参数。

在 Graph 的节点中，我们已经使用了这个装饰器：

```python
@traceable
@capture_dataset(prompt_name="parameter_parser", dataset_name="parameter_parser")
def parse_parameters_node(self, state: ReportState):
    # ... 节点逻辑 ...
    capture_inputs(user_inputs)  # 标记要捕获的数据
    # ... 调用 LLM ...
```

让我们在独立函数中演示如何使用：


In [ ]:
from langsmith import traceable
from tools.capture import capture_dataset, capture_inputs
from prompts.prompt_manager import PromptManager
from config.azure_config import AzureConfig

# 使用 @capture_dataset 装饰器（auto_sync=False 表示只保存到本地）
@traceable(name="test_parameter_parser_with_capture")
@capture_dataset(prompt_name="parameter_parser", dataset_name="param_parser_tests", auto_sync=False)
def test_param_parser_with_capture(user_query: str):
    """测试参数解析并自动捕获数据"""
    
    # 获取提示词
    prompt_manager = PromptManager()
    config = prompt_manager.get('parameter_parser')
    prompt = prompt_manager.create_prompt(config)
    
    # 准备输入
    inputs = {"user_query": user_query}
    
    # 🔑 关键：调用 capture_inputs 标记要捕获的数据
    capture_inputs(inputs, metadata={"source": "tutorial_demo"})
    
    # 调用 LLM
    llm = AzureConfig.get_llm()
    chain = prompt | llm
    result = chain.invoke(inputs)
    
    return result.content

# 测试多个查询
test_queries = [
    "写一份人工智能行业2024年发展报告",
    "分析金融科技2024年趋势，要求正式风格，深度分析",
    "云计算市场2024年简要分析"
]

print("🚀 开始测试并捕获数据...\n")

for i, query in enumerate(test_queries, 1):
    print(f"[{i}/{len(test_queries)}] 测试: {query}")
    result = test_param_parser_with_capture(query)
    print(f"✓ 完成")
    print(f"  解析结果预览: {result[:80]}...\n")

print("="*60)
print("✓ 所有测试完成！数据已自动捕获到本地")


---

### 5.4 同步数据到 LangSmith Dataset

将本地捕获的数据批量同步到 LangSmith Dataset，用于后续评估：


In [ ]:
# 同步到 LangSmith
from tools.capture import sync_all_captures

try:
    stats = sync_all_captures(dataset_filter="param_parser_tests")
    
    if stats['synced'] > 0:
        print(f"\n✓ 同步完成")
        print(f"  新同步: {stats['synced']} 条")
        print(f"  总计: {stats['total']} 条")
        print(f"\n📊 访问 LangSmith 查看 Dataset:")
        print(f"   https://smith.langchain.com/datasets")
    else:
        print(f"\n💡 提示:")
        print(f"  没有新数据需要同步")
        print(f"  请先运行上面的捕获演示")
        
except Exception as e:
    print(f"❌ 同步失败: {e}")
    print(f"\n💡 可能的原因:")
    print(f"  1. 还没有运行数据捕获")
    print(f"  2. LangSmith API Key 未配置")
    print(f"  3. 网络连接问题")


---

### 5.5 在 Graph 节点中使用捕获（回顾）

在 Graph 的节点实现中（`graph/nodes.py`），我们已经集成了捕获功能：

```python
@traceable
@capture_dataset(prompt_name="parameter_parser", dataset_name="parameter_parser")
def parse_parameters_node(self, state: ReportState):
    user_inputs = {"user_query": state.get("user_query", "")}
    capture_inputs(user_inputs, metadata={"prompt_version": ...})
    # ... 调用 LLM ...
```

这意味着每次运行 Graph 时，都会自动捕获真实的调用数据。

**完整流程：**
1. 运行 Graph → 自动捕获数据 → 保存到本地
2. 批量同步 → 推送到 LangSmith Dataset
3. 评估提示词 → 使用 Dataset 中的真实数据
4. 迭代优化 → 继续收集新的数据


In [ ]:
# 验证：在评估中使用捕获的数据
from prompts.prompt_manager import PromptManager
from evaluation.evaluators.report import ReportEvaluators

print("💡 提示：在评估提示词时，PromptManager 会自动使用 Dataset 中的数据进行评估")
print("\n例如，当我们运行评估时：")
print("  result = prompt_manager.evaluate_prompt('parameter_parser')")
print("\n评估器会使用 LangSmith Dataset 中的测试用例，包括我们捕获的真实数据")


---

## 🎓 总结与最佳实践

本教程展示了 LangSmith Prompt Pipeline 的完整工作流：

1. **提示词开发与迭代**（第3章）- 测试、评估、迭代的完整流程
2. **Graph 复杂场景**（第4章）- 理解多节点场景中的数据跟踪挑战
3. **Dataset 捕获**（第5章）- 系统性地收集和管理真实使用场景的数据

---

### 完整工作流

```mermaid
graph LR
    A[测试提示词] --> B[评估质量]
    B --> C[迭代优化]
    C --> D{质量达标?}
    D -->|是| E[推送到Hub]
    D -->|否| C
    E --> F[运行Graph]
    F --> G[自动捕获数据]
    G --> H[同步到Dataset]
    H --> B
```


In [1]:
### 最佳实践

1. **开发阶段**
   - 使用 `@capture_dataset` 自动收集真实数据
   - 定期同步到 LangSmith Dataset
   
2. **评估阶段**
   - 使用多个评估器全面评估
   - 设置合理的质量阈值（如 80%）
   
3. **发布阶段**
   - 评估通过后再推送到 Hub
   - 启用版本备份，便于回滚
   
4. **监控阶段**
   - 保持 LangSmith 追踪开启
   - 定期查看追踪数据和评估结果


SyntaxError: invalid character '（' (U+FF08) (2162190652.py, line 9)

### 快速参考

```python
# 1. 测试提示词
prompt_manager = PromptManager()
config = prompt_manager.get('parameter_parser')
prompt = prompt_manager.create_prompt(config)
chain = prompt | llm
result = chain.invoke(inputs)

# 2. 评估提示词质量
result = prompt_manager.evaluate_prompt("parameter_parser")

# 3. 迭代优化后推送
prompt_manager.push("parameter_parser", with_test=True, create_backup=True)

# 4. 运行 Graph（自动捕获数据）
builder = ReportGraphBuilder()
result = builder.run(user_query="...")

# 5. 同步捕获的数据
sync_all_captures()

# 6. 使用捕获的数据评估
# PromptManager 会自动使用 Dataset 中的数据
```


### 下一步

- 📖 查看项目文档了解更多功能
- 🔧 在实际项目中应用这些最佳实践
- 📊 持续收集和优化提示词质量

---

**💡 提示：** 本教程的所有功能示例都可以在前面的章节中找到详细演示。
